# Make Pokemon SV Datasets

In [1]:
import json
import pprint
from PIL import Image, ImageFilter
from PIL import ImageDraw

import glob
import re
import os
from loguru import logger
from tqdm import tqdm
#from tqdm import tqdm_notebook as tqdm
import numpy as np

import cv2
import sys
import shutil

import numpy as np
import matplotlib.pyplot as plt



## Setting param

データセットのパスや動画のフォルダなどのパラメーターを設定します．

In [2]:
root_path = "/home/Pokemon-SV"
datasets_root = "/home/Pokemon-SV-Datasets"

capture_dir = "capture"
capture_video_dir = "video"
capture_image_dir = "image"

capture_video_path = datasets_root  + "/" + capture_dir + "/" + capture_video_dir
capture_image_path = datasets_root  + "/" + capture_dir + "/" + capture_image_dir

diff_image_th1 = 0.50 * 1e9
diff_image_th2 = 0.85 * 1e9
save_freq     = 100
skip_freq     = 100

datasets_dir  = "datasets"
datasets_ver  = "v2"
datasets_path = datasets_root  + "/" + datasets_dir + "/" + datasets_ver
os.makedirs(datasets_path, exist_ok=True)

anotate_full = datasets_root  + "/" + datasets_path + "/result.json"
anotate_full_repath = datasets_root  + "/" + datasets_path + "/result_repath.json"

anotate_train_name = "pokemon_sv_train.json"
anotate_train_path = datasets_path + "/" + anotate_train_name
anotate_valid_name = "pokemon_sv_valid.json"
anotate_valid_path = datasets_path + "/" + anotate_valid_name

image_full_dir  = "images"
image_train_dir = "train2017"
image_valid_dir = "val2017"

diff_list_total = []

In [3]:
%cd $root_path

/home/Pokemon-SV


In [4]:
!ls

Dockerfile	    README.md  capture		   docs      utils
Make_Pokemon_SV.md  _datasets  docker-compose.yml  notebook


## パラメーター探索

動画の変化量のパラメータを検索します．


In [5]:
def param_analysis_video(video_path):
    
    video_name = video_path.split("/")[-1]
    video_single_path = capture_image_path + "/" + video_name
    logger.info("{:>20} : {}".format("video_single_path", video_single_path))
    os.makedirs(video_single_path, exist_ok=True)
       
    
    cap = cv2.VideoCapture(video_path)
    #total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frame_count = 100
    
    count = 0
    image_id = 1
    
    diff_list = []
    image_pix_list = []
    
    #while True:
    for _ in tqdm(range(total_frame_count)):
        ret, frame = cap.read()

        # 読み込み可能かどうか判定
        if ret:
            #logger.info("========================")
            #logger.info("{:>20} : {}".format("count", count))
            
            # 0番目は pre frameに登録のみで処理はskip
            if(count==0):
                pre_frame = frame
            else:
                # 0番目以降は処理
                
                image_pix_list.append(np.sum(np.abs(frame)))
                # 差分を計算
                diff_image = np.sum(np.abs(pre_frame - frame))
                #logger.info("{:>20} : {}".format("diff_image", diff_image))
                diff_list.append(diff_image)
                pre_frame = frame
                
                
            count += 1
        else:
            logger.info("Video Fin ...")
            break
            
    return diff_list, image_pix_list

In [6]:
def param_analysis_video_section():
    for video_path in video_list:
        logger.info("{:>20} : {}".format("video_path", video_path))
        diff_list, image_pix_list = param_analysis_video(video_path)
    
    return diff_list, image_pix_list

In [7]:
#diff_list_total, image_pix_list_total = param_analysis_video_section()

In [8]:
if(len(diff_list_total) > 0):
    plt.figure(figsize=[20,4.2])
    plt.hist(diff_list_total, bins=30)

## キャプチャー動画の分解

キャプチャーした動画を分解して画像に変換します．

変換のない静止した状態の画像はスキップした上で，`save_freq`フレームごとに画像を保存します．


### キャプチャー動画のリストを取得

In [9]:
glob_path = capture_video_path + "/*.mp4"
video_list = glob.glob(glob_path, recursive=True)
pprint.pprint(video_list)

['/home/Pokemon-SV-Datasets/capture/video/2022-11-19 20-26-07.mp4']


### 動画の分解と保存

In [10]:
def analysis_video(video_path):
    
    video_name = video_path.split("/")[-1]
    video_single_path = capture_image_path + "/" + video_name
    logger.info("{:>20} : {}".format("video_single_path", video_single_path))
    os.makedirs(video_single_path, exist_ok=True)
       
    
    cap = cv2.VideoCapture(video_path)
    #total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frame_count = 2000
    
    count = 0
    image_id = 0
    count_save = 0
    diff_list1 = []
    diff_list2 = []
    
    
    #while True:
    for _ in tqdm(range(total_frame_count)):
        ret, frame = cap.read()

        # 読み込み可能かどうか判定
        if ret:
            #logger.info("========================")
            #logger.info("{:>20} : {}".format("count", count))
            
            # 0番目は pre frameに登録のみで処理はskip
            if(count==0):
                pre_frame = frame
            else:
                # 0番目以降は処理
                
                if(count % skip_freq == 0):
                
                    # 差分を計算
                    diff_image = np.sum(np.abs(pre_frame - frame))


                    # 閾値以上なら処理する
                    if(diff_image > diff_image_th1):

                        save_image_name = "{:09d}.jpg".format(image_id)
                        save_image_path = video_single_path + "/" + save_image_name
                        #logger.info("{:>20} : {}".format("frame[pix]", np.sum(np.abs(frame))))
                        #logger.info("{:>20} : {}".format("save_image_path", save_image_path))
                        cv2.imwrite(save_image_path, frame)

                        pre_save_frame = frame.copy()
                        image_id += 1
                        
                pre_frame = frame
                
                
            count += 1
        else:
            logger.info("Video Fin ...")
            break
        
    return diff_list1, diff_list2
        

In [11]:
def video_section():
    for video_path in video_list:
        logger.info("{:>20} : {}".format("video_path", video_path))
        diff_list1, diff_list2 = analysis_video(video_path)
        return diff_list1, diff_list2

In [12]:
diff_list1, diff_list2 = video_section()

2022-11-25 09:47:55.993 | INFO     | __main__:video_section:3 -           video_path : /home/Pokemon-SV-Datasets/capture/video/2022-11-19 20-26-07.mp4
2022-11-25 09:47:55.995 | INFO     | __main__:analysis_video:5 -    video_single_path : /home/Pokemon-SV-Datasets/capture/image/2022-11-19 20-26-07.mp4
100%|██████████| 2000/2000 [00:07<00:00, 264.01it/s]


In [13]:
if(len(diff_list1) > 0):
    plt.figure(figsize=[20,4.2])
    plt.hist(diff_list1, bins=30)

In [14]:
if(len(diff_list2) > 0):
    plt.figure(figsize=[20,4.2])
    plt.hist(diff_list2, bins=30)

## 画像をアノテーション

こちらのアノテーションソフトを使ってアノテーションしていきます．

https://github.com/makiMakiTi/label-studio-1.6.0

下記のコマンドにて実行可能です．

```bash
docker-compose up --build
```

## アノテーションファイルの修正

exportされたアノテーションファイル`datasets\v0\result.json`は画像のパスが`COCO`フォーマットになっていないので修正します．



読み込みます

In [15]:
with open(anotate_full, 'rt', encoding='UTF-8') as annotations:
    result_coco = json.load(annotations)

FileNotFoundError: [Errno 2] No such file or directory: '/home/Pokemon-SV-Datasets//home/Pokemon-SV-Datasets/datasets/v2/result.json'

パスを修正しファイル名にします．

In [ ]:
for i in range(len(result_coco["images"])):
    file_name = result_coco["images"][i]['file_name']    
    result_coco["images"][i]['file_name'] = file_name.split("/")[-1]

書き出します．

In [ ]:
with open(anotate_full_repath, 'wt', encoding='UTF-8') as coco:
        json.dump(result_coco, coco, indent=2, sort_keys=True)

## データセットの split

データセットの分割します．

In [ ]:
!python utils/cocosplit.py --having-annotations --multi-class -s 0.8 $anotate_full_repath $anotate_train_path $anotate_valid_path

In [ ]:
def move_datasets_image_file(target_dir, anno_path):
    
    logger.info("{:>20} : {}".format("target_dir", target_dir))
    logger.info("{:>20} : {}".format("anno_path", anno_path))
    os.makedirs(target_dir, exist_ok=True)
    
    with open(anno_path, 'rt', encoding='UTF-8') as annotations:
        result_coco = json.load(annotations)

    for i in tqdm(range(len(result_coco["images"]))):
        #logger.info(">>>>>>>>>>>> {:>20} : {}".format("i", i))
        
        file_name = result_coco["images"][i]['file_name']   
        #logger.info("{:>20} : {}".format("file_name", file_name))
        
        source_path =  datasets_path + "/" + image_full_dir + "/" + file_name
        #logger.info("{:>20} : {}".format("source_path", source_path))
        
        target_path =  target_dir + "/" + file_name
        #logger.info("{:>20} : {}".format("target_path", target_path))
        
        shutil.copyfile(source_path, target_path)
        
    #pprint.pprint(result_coco)

In [ ]:
move_datasets_image_file(target_dir=datasets_path + "/" + image_train_dir, anno_path=anotate_train_path)

In [ ]:
move_datasets_image_file(target_dir=datasets_path + "/" + image_valid_dir, anno_path=anotate_valid_path)

In [ ]:
c_list = []
for c in result_coco['categories']:
    print(c['name'])
    c_list.append(c['name'])
    
c_tuple = tuple(c_list)
    

In [ ]:
c_tuple